In [1]:
from lxml import html
import requests
import time
import re
import cssselect

In [2]:
prob_url = "https://www.fangraphs.com/livescoreboard.aspx?date={date}".format(date = time.strftime("%Y-%m-%d"))
prob_page = requests.get(prob_url)
prob_tree = html.fromstring(prob_page.content)

In [3]:
prob_teams = []
for a in prob_tree.xpath('//tr//td//a[contains(@href, "winss.aspx")]'):
    prob_teams.append(re.search(r"team=([a-zA-Z ]*)&", str([a.attrib['href']])).group(1).lower())
prob_odds = prob_tree.xpath('//center/table//tr/td[@align="center"]/text()')

In [4]:
probs_dict = {}
i=0
while i < len(prob_odds):
    probs_dict[prob_teams[i]]=float(prob_odds[i][:4])
    i+=1
print(probs_dict)

{'orioles': 50.3, 'white sox': 49.7, 'diamondbacks': 52.6, 'brewers': 47.4, 'tigers': 38.9, 'twins': 61.1, 'padres': 34.3, 'nationals': 65.7, 'yankees': 65.3, 'rangers': 34.7, 'marlins': 40.0, 'mets': 60.0, 'braves': 45.9, 'phillies': 54.1, 'royals': 36.1, 'cardinals': 63.9, 'rockies': 38.3, 'dodgers': 61.7}


In [5]:
betting_page = requests.get('http://www.vegasinsider.com/mlb/odds/las-vegas/')
betting_tree = html.fromstring(betting_page.content)
bet_teams = []
for a in betting_tree.xpath('//td[@class="viBodyBorderNorm"]//td//b/a[contains(@href, "/team/")]'):
    bet_teams.append(re.search(r"team\/([a-zA-Z -]+)", str([a.attrib['href']])).group(1).lower().replace('-', ' '))
bet_odds = betting_tree.xpath('//td[@class="viBodyBorderNorm"]//td[3]//a[contains(@href, "/line-movement/")]/text()')

In [6]:
total_odds = {}
i=0
j=0
while i < len(bet_teams):
    if i%3:
        if bet_teams[j] not in total_odds.keys():
            total_odds[bet_teams[j]]=bet_odds[i]
            j+=1
    i+=1

{'padres': '+165', 'nationals': '-180\n\t\t\t\t\t\t\t', 'braves': '-101', 'phillies': '-109\n\t\t\t\t\t\t\t', 'marlins': '+120', 'mets': '-130\n\t\t\t\t\t\t\t', 'diamondbacks': '-115', 'brewers': '+105\n\t\t\t\t\t\t\t', 'rockies': '+160', 'dodgers': '-180\n\t\t\t\t\t\t\t', 'yankees': '-205', 'rangers': '+185\n\t\t\t\t\t\t\t', 'orioles': '-105', 'white sox': '-105\n\t\t\t\t\t\t\t', 'tigers': '+185', 'twins': '-205\n\t\t\t\t\t\t\t', 'royals': '+180', 'cardinals': '-200\n\t\t\t\t\t\t\t'}


In [40]:
class Game:
    "A class holding opponents, odds, probabilities, and any other stat concerning the two teams playing a certain game"
    def __init__(self, team1, team2, team1odds, team2odds, team1winpct, team2winpct):
        self.team1 = team1
        self.team2 = team2
        self.team1odds = team1odds
        self.team2odds = team2odds
        self.team1winpct = team1winpct
        self.team2winpct = team2winpct
    
    def risk_team1(self):
        if float(self.team1odds) > 0:
            win_total = self.team1winpct/100*(self.team1odds/100 +1) -1
        else:
            win_total = self.team1winpct*(1/100 - 1/self.team1odds) - 1
        return win_total
    
    def risk_team2(self):
        if float(self.team1odds) > 0:
            win_total = self.team2winpct/100*(self.team2odds/100 +1) -1
        else:
            win_total = self.team2winpct*(1/100 - 1/self.team2odds) - 1
        return win_total

            

In [41]:
i=0
games={}
while i < len(prob_odds):
    key = 'game_'+str(i/2 + 1)
    games[key]=Game(prob_teams[i], 
                    prob_teams[i+1], 
                    int(total_odds[prob_teams[i]]), 
                    int(total_odds[prob_teams[i+1]]), 
                    float(prob_odds[i][:4]), 
                    float(prob_odds[i+1][:4]))
    i+=2
print(games)

{'game_1.0': <__main__.Game object at 0x110c8bda0>, 'game_2.0': <__main__.Game object at 0x110c8bd68>, 'game_3.0': <__main__.Game object at 0x110c8bd30>, 'game_4.0': <__main__.Game object at 0x110c8bcf8>, 'game_5.0': <__main__.Game object at 0x110c8bcc0>, 'game_6.0': <__main__.Game object at 0x110c8bc88>, 'game_7.0': <__main__.Game object at 0x110c8bc50>, 'game_8.0': <__main__.Game object at 0x110c8bc18>, 'game_9.0': <__main__.Game object at 0x110c8bbe0>}


In [42]:
print(games['game_1.0'].team1, games['game_1.0'].team1odds, games['game_1.0'].team1winpct, games['game_1.0'].risk_team1())

orioles -105 50.3 -0.01795238095238083


In [43]:
print(games['game_1.0'].team2, games['game_1.0'].team2odds, games['game_1.0'].team2winpct, games['game_1.0'].risk_team2())

white sox -105 49.7 -0.029666666666666508


In [46]:
for key in games:
    print(games[key].team1, games[key].risk_team1(), games[key].team2, games[key].risk_team2())

orioles -0.01795238095238083 white sox -0.029666666666666508
diamondbacks -0.01660869565217382 brewers -0.9774285714285714
tigers 0.10865000000000014 twins -1.6415499999999998
padres -0.09105000000000008 nationals -1.5256
yankees -0.028463414634146478 rangers -0.8405675675675676
marlins -0.11999999999999988 mets -1.18
braves -0.08654455445544551 phillies 0.037330275229358056
royals 0.01079999999999992 cardinals -1.639
rockies -0.0042000000000000925 dodgers -1.4936
